# Linear Classifiers in Python

#### Logistic Regression
* Logistic Regression is a linear classifier
* sklearn's Logistic Regression can also output confidence scores rather than "hard" or definite predictions with `.predict_proba()`

```
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.predict(X_test)
lr.score(X_test, y_test)
```

#### Using Linear SVC
* In sklearn the basic SVM classifier is called `LinearSVC()` or Linear Support Vector Classifier
* Note that sklearn's Logistic Regression and SVM implementations handle multiple classes (if a dataset has more than 2 classes) automatically.

#### Using SVC
* The SVC class fits a nonlinear SVM by default

* **Underfitting:** model is too simple, training accuracy low
* **Overfitting:** model is too complex, testing accuracy low

#### Linear Decision Boundaries
* A decision boundary tells us what class our classifier will predict for any value of x
* A decision boundary is considered **linear** when it is a line (in any orientation)
    * This definition extends to (classifying) more than 2 features
    * For five features, the space of possible x-values would be five-dimensional. In this case, the boundary would be a higher-dimensional **hyperplane** cutting the space into two halves.
* A **nonlinear** boundary is any other type of boundary.
    * Sometimes this leads to non-contiguous regions regions of a certain prediction ("islands", etc).
* In their basic forms, logistic regression and SVMs are linear classifiers, which means they learn linear decision boundaries.
    * However in some more complex forms, both may learn nonlinear decision boundaries

#### Vocabulary:
* **Classification:** learning to predict categories
* **Regression:** learning to predict a continuous value
* **Decision boundary:** the surface separating different predicted classes
* **Linear classifier:** a classifier that learns linear decision boundaries 
    * e.g. logistic regression, linear SVM
* **Linearly separable:** A data set is called linearly separable if it can be perfectly explained by a linear classifier **(straight line)**